<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li>

# Проект для «Викишоп»

## Цель проекта

Цель данного проекта заключается в создании прототипа модели машинного обучения, способной классифицировать комментарии на позитивные и негативные в интернет-магазине "Викишоп". Эта модель поможет автоматически выявлять токсичные комментарии и отправлять их на модерацию. Такой инструмент позволит поддерживать безопасное и уважительное взаимодействие в сообществе клиентов.

## Структура проекта

Проект разделен на три основные части:

1. Обработка и анализ данных

- Подробное описание исходных данных и их структуры.
- Предварительный анализ данных для понимания основных характеристик.
- Визуализация данных с целью лучшего понимания распределения комментариев.
2. Обучение моделей

- Введение в разработку и обучение моделей классификации комментариев.
- Декомпозиция задачи на локальные задачи для более эффективного обучения.
- Методология и методы обучения моделей.
- Подбор и обучение различных моделей для достижения значения метрики качества F1 не менее 0.75.
3. Подведение итогов

- Выводы из анализа данных и обучения моделей.
- Оценка результатов моделирования и их интерпретация.
- Рекомендации на основе полученных результатов для улучшения модели или более точной классификации токсичных комментариев.

## Подготовка

Перед началом работы загрузим все необходимые нам в дальнейшем библиотеки и дополнения

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from tqdm import notebook
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.dummy import DummyClassifier

import warnings
warnings.filterwarnings('ignore')

Сохраним в отдельной переменной данные из нашего файла и откроем их. Первый столбец из начальных данных возьмем в качестве индексов, так как эти данные для обучения и предсказания нам не понядобятся

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
display(data.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим данные на корректность их типа, наличие дубликатов и пропусков.

In [3]:
print(data.info());
print()
print('Количество дубликатов в данных:', data.duplicated().sum())
print()
print('Количество пропусков в данных:')
print(data.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None

Количество дубликатов в данных: 0

Количество пропусков в данных:
text     0
toxic    0
dtype: int64


Создадим функцию для лемматизации данных, удалим цифры, лишние символовы в том числе лишние пробелы внутри функции.

In [4]:
def text_lemm(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    joined_text = ' '.join(lemmatized_tokens)
    text_only = re.sub(r'[^a-zA-Z]', ' ', joined_text)
    result = ' '.join(text_only.split())
    return result

Так как в наших данных комментарии написанны на иностранном языке, а именно на английском, загружаем библиотеку слов-слов английского языка и преобразовываем его в отдельный набор. 

In [5]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Теперь удалим весь "мусор" из наших комментариев и сохраним чистые данные в отдельный столбец

In [6]:
notebook.tqdm.pandas() 
def text_preprocessing(text):
    lemmatized_text = text_lemm(text)
    words = lemmatized_text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

data['clear_text'] = data['text'].progress_apply(text_preprocessing)

  0%|          | 0/159292 [00:00<?, ?it/s]

Выделим в отдельные переменные наш целевой признак и признаки, а также разделим наши данные на тренировочную и тестовую выборки 75/25

In [7]:
features = data['clear_text']
target = data['toxic']

train_features, test_features, train_target, test_target = train_test_split(features, target,
                                                                            test_size=0.25,
                                                                            random_state=12345)
print(train_features.shape)
print(train_target.shape)
print(test_features.shape)
print(test_target.shape)

(119469,)
(119469,)
(39823,)
(39823,)


## Обучение

Выполним векторизацию текстовых данных с использованием метода TF-IDF для дальнейшего обучения моделей.

In [8]:
vectorizer = TfidfVectorizer()
train_features_tfidf = vectorizer.fit_transform(train_features)
test_features_tfidf = vectorizer.transform(test_features)

Проведем кросс-валидацию и обучение модели Логистической регрессии

In [9]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__max_features': [900],
    'classifier__C': [1.0, 5.0, 10.0],
    'classifier__class_weight': ['balanced']
}
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='f1')
grid_search.fit(train_features, train_target)
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print('Лучшие параметры:', best_params)
print('Лучший показатель F1:', grid_search.best_score_)

Лучшие параметры: {'classifier__C': 1.0, 'classifier__class_weight': 'balanced', 'tfidf__max_df': 0.25, 'tfidf__max_features': 900}
Лучший показатель F1: 0.5679246037957828


Следующим будет обучение и подбор параметров CatBoost

In [10]:
params = {
    'max_depth': [2, 3, 5], 
    'n_estimators': [100, 500, 600], 
    'random_state': [12345], 
    'auto_class_weights': ['Balanced']
}

model_cbc = CatBoostClassifier(eval_metric='TotalF1')
train_pool = Pool(train_features_tfidf, train_target)
model_cbc_gs = model_cbc.grid_search(params, train_pool, verbose=True, cv=5, plot=True)

best_mean_f1 = max(model_cbc_gs['cv_results']['test-TotalF1-mean'])
best_params = model_cbc_gs['params']

print('Лучший результат F1-меры:', best_mean_f1)
print('Лучшие параметры:', best_params)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.5356546	test: 0.5388601	best: 0.5388601 (0)	total: 671ms	remaining: 1m 6s
1:	learn: 0.5483525	test: 0.5425665	best: 0.5425665 (1)	total: 1.26s	remaining: 1m 1s
2:	learn: 0.5483525	test: 0.5425665	best: 0.5425665 (1)	total: 1.83s	remaining: 59.3s
3:	learn: 0.5483525	test: 0.5425665	best: 0.5425665 (1)	total: 2.42s	remaining: 58.1s
4:	learn: 0.5483525	test: 0.5425665	best: 0.5425665 (1)	total: 3.03s	remaining: 57.5s
5:	learn: 0.5483525	test: 0.5425665	best: 0.5425665 (1)	total: 3.63s	remaining: 56.9s
6:	learn: 0.5485022	test: 0.5428918	best: 0.5428918 (6)	total: 4.2s	remaining: 55.9s
7:	learn: 0.6066828	test: 0.6060933	best: 0.6060933 (7)	total: 4.79s	remaining: 55.1s
8:	learn: 0.6102446	test: 0.6085264	best: 0.6085264 (8)	total: 5.4s	remaining: 54.6s
9:	learn: 0.6102446	test: 0.6085264	best: 0.6085264 (8)	total: 6s	remaining: 54s
10:	learn: 0.6098897	test: 0.6085456	best: 0.6085456 (10)	total: 6.64s	remaining: 53.7s
11:	learn: 0.6070541	test: 0.6066820	best: 0.6085456 (10)	t

Третьей моделью будет Случайный лес

In [11]:
from sklearn.model_selection import cross_val_score

estimators = list(range(1, 12, 4))
param_grid = {'max_depth': [10, 15, 20], 
          'n_estimators': [50, 100, 150], 
          'random_state': [12345], 
          'class_weight': ['balanced']
             }
model_rf = RandomForestClassifier(random_state=12345)

grid_search = GridSearchCV(model_rf, param_grid, cv=5, scoring='f1')
grid_search.fit(train_features_tfidf, train_target)

best_model_rf = grid_search.best_estimator_
best_result_rf = grid_search.best_score_
best_depth_rf = best_model_rf.n_estimators

print('F1 наилучшей модели равно:', best_result_rf.round(2), end='')
print(' C глубиной:', best_depth_rf)


F1 наилучшей модели равно: 0.39 C глубиной: 150


Мы провели обучение трех моделей и получили следующие результаты по метрике F1:

- Логистическая регрессия = 0.57

- CatBoost = 0.86

- Случайный лес = 0.39

Лучший результат нам дал CatBoost, также он единственный из трех моделей прошел обозначенный порог  >0.75 

Проведем тестирование выбранной намимодели на тестовой выборке и посмотрим пройдет ли она наш порог.

In [12]:
best_model = CatBoostClassifier(eval_metric='TotalF1', **best_params)
best_model.fit(train_features_tfidf, train_target)

test_predictions = best_model.predict(test_features_tfidf)
test_f1_score = f1_score(test_target, test_predictions)

print('Результат F1-меры на тренировочной выборке:', test_f1_score)

Learning rate set to 0.12687
0:	learn: 0.6111823	total: 1.87s	remaining: 18m 41s
1:	learn: 0.6112133	total: 3.47s	remaining: 17m 17s
2:	learn: 0.6529755	total: 5.12s	remaining: 16m 59s
3:	learn: 0.6555072	total: 6.7s	remaining: 16m 37s
4:	learn: 0.6892525	total: 8.17s	remaining: 16m 12s
5:	learn: 0.7096946	total: 9.64s	remaining: 15m 54s
6:	learn: 0.7197614	total: 11.1s	remaining: 15m 43s
7:	learn: 0.7357545	total: 12.7s	remaining: 15m 36s
8:	learn: 0.7436634	total: 14.2s	remaining: 15m 31s
9:	learn: 0.7544099	total: 15.8s	remaining: 15m 30s
10:	learn: 0.7604512	total: 17.2s	remaining: 15m 22s
11:	learn: 0.7629251	total: 18.7s	remaining: 15m 17s
12:	learn: 0.7674947	total: 20.2s	remaining: 15m 12s
13:	learn: 0.7691749	total: 21.7s	remaining: 15m 6s
14:	learn: 0.7694562	total: 23.1s	remaining: 15m 2s
15:	learn: 0.7780874	total: 24.6s	remaining: 14m 59s
16:	learn: 0.7786408	total: 26.2s	remaining: 14m 57s
17:	learn: 0.7800208	total: 27.6s	remaining: 14m 54s
18:	learn: 0.7845710	total: 29

Также проверим модель на адекватность при помощи дамми модели.

In [13]:
dummy_model = DummyClassifier(strategy='uniform', random_state=12345)
dummy_model.fit(train_features_tfidf, train_target)
print(f1_score(test_target, dummy_model.predict(test_features_tfidf)))

0.16900237035804883


## Выводы

Мы подготовили текстовые данные и обучили на них три модели : Логистическая регрессия, Случайный лес и CatBoost.Только последняя соответствовала нашему условию F1 > 0.75. Так как в данном случае от нас не требовалось выбрать модель с наименьшим временем обучения и предсказания, то мы выбрали модель с самыми высокими показателями метрики. На тестовой выборке показатели метрики этой модели также соответствуют условию 0.756 > 0.75